In [1]:
import glob
import slab
from slab import *
import numpy as np
import subprocess
import time
import os
from h5py import File, special_dtype
import gc
import yaml
from datetime import datetime, timedelta
from slab import get_next_filename, Experiment
from slab.instruments import RGA100, XLN10014

import threading
from threading import Lock 

In [2]:
rga=RGA100(address='COM9')

In [3]:
# rga.set_filament_off()
# rga.get_filament_state()

In [4]:
class ThreadHandler():
    def __init__(self, jobs=None, max_threads=1):
        if jobs==None:
            self.jobs=[]
        else:
            self.jobs=jobs
        self.thread_lock=Lock()

    def _check_for_alive(self, jobs=None):
        if jobs==None:
            jobs=self.jobs

        while False:
            self.thread_lock.acquire(blocking=False)

        with self.thread_lock:
            current_ident=threading.get_ident()
            alive_jobs=[]
            if jobs==[]:
                pass
            else:
                for I, job in enumerate(jobs):
                    if job._ident==current_ident:
                        pass
                    else:
                        if job.is_alive()==True:
                            alive_jobs.append(job)
                        elif job.is_alive()==False:
                            pass
            if len(alive_jobs)>0:
                state=True
            else:
                state=False
        return state

    def _get_job_names(self):
        names=[]
        if self.jobs==[]:
            pass
        else:
            for job in self.jobs:
                if job.is_alive():
                    names.append(job._name)
                else:
                    pass
        return names

    def _set_thread(self, name, func, *args):
        self._del_dead_threads()
        thread = threading.Thread(name=self._thread_name_check(name), target=func, args=args)
        self.jobs.append(thread)
        self.num_threads+=1
        return thread

    def _thread_name_check(self, name):
        for jobs in self.jobs:
            if jobs._name==name:
                name_split=name.split('_')
                if len(name_split)==1:
                    name=name+'_%i'%(1)
                else:
                    name=name_split[0]+'_'+str(int(name_split[1])+1)
                name=self._thread_name_check(name)
            else:
                name=name
        return name

    def _del_dead_threads(self):
        if self.jobs==[]:
            pass
        else:
            for I, job in enumerate(self.jobs):
                if job.is_alive()==False:
                    del(self.jobs[I])
        self.num_threads=len(self.jobs)

    def _thread_check(self):
        self._del_dead_threads()
        while self.num_threads>=self.max_threads:
            self._del_dead_threads()


class vacuum(ThreadHandler):
    def __init__(self, syst_config='S:\\_Furnace\\Automation Scripts\\system_config.yml'):

        super(vacuum, self).__init__(jobs=None, max_threads=4)

        try:
            self.im=InstrumentManager()
        except: 
            raise Exception('Unable to open Instrument Manager Server')

        try:
            self.AG850 = self.im.AG850
        except:
            raise Exception('Unable to connect to turbo controller')
        try:
            self.P_monitor = self.im.pressure_gauge
        except:
            raise Exception('Unable to connect to pressure gauge')
        try:
            self.PID = self.im.PID
        except:
            raise Exception('Unable to connect to PID heaters')
        try:
            self.PLC = self.im.PLC
        except:
            raise Exception('Unable to connect to PLC')
        self.config = syst_config

        with open(self.config, 'r') as f:
            self.v_config = yaml.full_load(f)['PLC']['VALVES']

        self.valves = {}
        for chs in self.v_config:
            self.valves[chs] = {'CHANNEL': self.v_config[chs]['CHANNEL'], 'STATE': 0}
        
        self.VAC_jobs=[]
        self.VAC_lock=Lock()

        self.get_valves()
        self.start_pos = [0] * 8
        self.valve_pos = self.curr_pos.copy()
        # self.PID.load_config(syst_config)

    def load_p_monitor_config(self):
        if self._check_for_alive(jobs=self.VAC_jobs):
            return 
        print('System Config: %s' % (self.config))
        try:
            self.P_monitor.load_config(self.config)
            print('________________________________')
            print('Pressure Monitor Parameters set:')

        except:
            print('Unable to Load Pressure Config')

    def load_PID_config(self):
        print('System Config: %s' % (self.config))
        try:
            self.PID.load_config(self.config)
            print('________________________________')
            print('PID Heater Parameters set:')
        except:
            print('Unable to Load PID Config')

    def load_turbo_config(self):
        if self._check_for_alive(jobs=self.VAC_jobs):
            return 
        print('System Config: %s' % (self.config))
        try:
            self.AG850.load_config(self.config)
            print('________________________________')
            print('Turbo Parameters set:')
        except:
            print('Unable to Load Turbo Config')

    def full_start(self, heaters=False):
        if self._check_for_alive(jobs=self.VAC_jobs):
            return 
        pump_stat = self.AG850.get_status()
        if pump_stat == 'stop':
            self.set_open('BACKING')
            self.set_open('ROUGHING')
            self.set_open('GATE')
            self.set_open('RGA')
            self.set_close('PULSE')
            self.set_valves()
            print('________________________________')
            print('Valves set, starting backing pump')
            self.AG850.set_purge(state=1)
            timeout = 3600.00
            start_time = time.time()
            elapsed_time = time.time() - start_time
            while self.P_monitor.set_point_status()[4] == 0 and elapsed_time <= timeout:
                elapsed_time = time.time() - start_time
                pass
            if elapsed_time >= timeout:
                print('Process timed out, unable to reach roughing pressure')
            else:
                pass
            self.AG850.start()
            print('________________________________')
            print('Turbo Started')
            while self.AG850.get_freq() >= 60.0:
                pass
            while self.AG850.get_freq() <= 100.00:
                pass
            self.set_close('ROUGHING')
            self.set_valves()
            print('________________________________')
            print('Initial Pumping Started')

            if heaters == True:
                while self.P_monitor.set_point_status()[5] == 0:
                    pass
                self.PID.start_all()
                print('________________________________')
                print('Bakeout Heaters Turned On')
        else:
            print('Turbo pump must be in "STOP" condition')


    def set_open(self, key):
        if self._check_for_alive(jobs=self.VAC_jobs):
            return 
        self.valves[key]['STATE'] = 1

    def set_close(self, key):
        if self._check_for_alive(jobs=self.VAC_jobs):
            return 
        self.valves[key]['STATE'] = 0

    def set_valves(self):
        if self._check_for_alive(jobs=self.VAC_jobs):
            return 
        new_valve_pos = [0] * len(self.curr_pos)
        for key in iter(self.valves.keys()):
            new_valve_pos[self.valves[key]['CHANNEL'] - 1] = self.valves[key]['STATE']
        set_inds = [index for index, elem in enumerate(new_valve_pos) if elem != self.curr_pos[index]]
        for inds in set_inds:
            self.PLC.set_valve(inds + 1, new_valve_pos[inds])

        self.get_valves();

    def _par_nitrogen_backfill(self, f_time):
        f_time=int((f_time-(f_time-60)%60)/60)
        if self._check_for_alive(jobs=self.VAC_jobs):
            return 
        self.set_close('RGA')
        self.set_valves()
        self.set_close('GATE')
        self.set_open('ROUGHING')
        self.set_valves()
        if self.PLC.get_state(5)[1]==False:
            self.PLC.set_pulse_time(f_time)
            self.PLC.pulse(self.valves['PULSE']['CHANNEL'])
            et=0
            start_time=time.time()
            while et<=f_time*60 or not self.P_monitor.set_point_status()[5]:
                et=time.time()-start_time
                pass
            self.set_open('GATE')
            self.set_close('ROUGHING')
            self.set_valves()
            while not self.P_monitor.set_point_status()[0]:
                pass
            time.sleep(.5)
            self.set_open('RGA')
            self.set_valves()

    def get_valves(self):
        while False: 
            self.VAC_lock.acquire(blocking=False)
        with self.VAC_lock:
            try:
                v_state = self.PLC.get_all()
                self.curr_pos = v_state
                return self.curr_pos
            except:
                raise Exception('ERROR occured in PLC, reset PLC')

    def cal_CTR(self):
        if self._check_for_alive(jobs=self.VAC_jobs):
            return 
        pres = self.P_monitor.p_read(1)[0]
        if pres < 2E-6:
            self.P_monitor.cal_ctr()
            print('________________________________')
            print('CTR calibrated')
        else:
            print('________________________________')
            print('Pressure must be below 2E-6 for accurate CTR calibration')

    def heaters_on(self, state=True, load_config=False):
        if load_config == True:
            self.load_PID_config()
        else:
            pass
        if state == True:
            self.PID.start_all()
            print('________________________________')
            print('Bakeout Heaters On')
        elif state == False:
            self.PID.stop_all()
            print('________________________________')
            print('Bakeout Heaters Off')

    def full_stop(self):
        if self._check_for_alive(jobs=self.VAC_jobs):
            return 
        pump_stat = self.AG850.get_status()
        self.PID.stop_all()
        self.get_valves()
        self.set_close('GATE')
        self.set_close('RGA')
        self.set_close('ROUGHING')
        self.set_valves()
        if pump_stat != 'stop':
            if pump_stat == 'normal':
                self.AG850.stop()
                print('Turbo Shut Off')
            elif pump_stat == 'starting':
                self.AG850.stop()
            print('________________________________')
            print('Turbo Shut Off')
        else:
            pass
        self.set_close('BACKING')
        self.set_valves()
        self.AG850.set_purge(0)
        print('________________________________')
        print('Backing Pump Shut Off')

    def get_pressure(self, ch):
        return self.P_monitor.p_read(ch)[0]

    def get_PID_temps(self):
        return self.PID.get_temp(0)

    def reset_plc(self):
        if self._check_for_alive(jobs=self.VAC_jobs):
            return
        self.PLC.reset()

class FurnaceSetup(vacuum):
    def __init__(self, config_path=None, system_config_path=None, kill_server=False, p_server=False):
        local_path=os.getcwd()
        if config_path==None:
            config_path=local_path+'\\furnace_config.yml'
        if system_config_path==None:
            system_config_path=local_path+'\\system_config.yml'

        try:
            super(FurnaceSetup, self).__init__(system_config_path)
        except:
            raise Exception('Unable to load vacuum interface, check system_config.yml is in same path or in specified path')

        try:
            with open(config_path, 'r') as f:
                self.furnace_config = yaml.full_load(f)['Furnace']
            with open(config_path, 'r') as f:
                self.RGA_config=yaml.full_load(f)['RGA']
            with open(config_path, 'r') as f:
                self.nitrogen_config = yaml.full_load(f)['Nitrogen_Doping']
        except:
            raise Exception('Unable to load furnace config parameters, check config param keys')
        try:
            with open(system_config_path, 'r') as f:
                self.supply_config=yaml.full_load(f)['SUPPLY']
        except:
            raise Exception('Unable to load supply config parameters, check config param keys')



        print('_________________________')
        for keys in iter(self.furnace_config.keys()):
            print(keys.replace("_", " ")+': '+str(self.furnace_config[keys]))
        print('_________________________')

        #RGA and logging parameters
        self.RGA_port=self.RGA_config['port']

        #supply safety params
        self.max_current = self.supply_config['max_current']
        self.max_volts = self.supply_config['max_volts']
        self.min_volts = self.supply_config['min_volts']
        self.supply_port = self.supply_config['port']

        #funace ramping and logging parameters
        self.ramp_up_rate = self.furnace_config['Ramp_up_rate']
        self.ramp_down_rate = self.furnace_config['Ramp_down_rate']
        self.P_hold = np.float(self.furnace_config['P_hold'])
        self.Hold_v = self.furnace_config['Hold_v']
        self.Hold_time = self.furnace_config['Hold_time']
        self.log_interval = self.furnace_config['logging_interval']
        self.PH_time=self.furnace_config['Pre_heat_time']
        self.P_start=np.float(self.furnace_config['P_start'])

        #nitrogen doping parameters
        self.nitrogen_state=self.nitrogen_config['nitrogen_step']
        self.nitrogen_start_time=self.nitrogen_config['start_time']
        self.nitrogen_hold_time=self.nitrogen_config['nitrogen_hold_time']
        self.nitrogen_end_time=self.nitrogen_start_time+self.nitrogen_config['nitrogen_hold_time']

        if self.nitrogen_end_time>self.Hold_time:
            print('_________________________')
            print('Nitrogen start and end times must be less than the total hold time. Exiting automation')
            exit()

        # if self.min_volts<=12.0:
        #     print('_________________________')
        #     print('Minimum voltage lower than allowable 12 volts, exiting automation')
        #     exit()

        if self.max_volts<=self.min_volts:
            print('_________________________')
            print('Maximum voltage less than minimum voltage, exiting automation')
            exit()

        if self.Hold_v<self.min_volts or self.Hold_v>self.max_volts:
            print('_________________________')
            print('Holding voltage not valid, must be less than max voltage and greater than min voltage')
            print('Exiting automation')
            exit()

        if self.P_start>=self.P_hold:
            print('_________________________')
            print('Degas pressure hold point must be greater than ramp restart pressure')
            print('Exiting automation')
            exit()

        self.kill_s = kill_server

        #############################################################
        # Server code, comment out if just debugging the automation #
        #############################################################
        if p_server==True:
           self.start_furnace_server()

        #############################################################
        #                    End of server code                     #
        #############################################################

        try:
            self.im = InstrumentManager()
        except:
            raise Exception('ERROR in contacting instrument manager server')

        try:
            self.xln = XLN10014(address=self.supply_port)
        except:
            raise Exception('ERROR in contacting XLN power supply')
        
        self.max_threads=4
        self.num_threads=0
        self.log_lock=Lock()
        self.thread_lock=Lock()
        self.jobs=[]

    def start_furnace_server(self):
        self.PS = Server()
        self.PID = self.PS.get_PID()
        self.p_log = self.PS.get_log()

        self.pp_keys = self.PS.pp_keys()

        local_time = time.time()
        self.st = datetime.fromtimestamp(local_time).strftime('%Y-%m-%d %H:%M:%S').split(' ')[0]

        prefix = self.st.split(' ')[0] + '_Furnace_log'
        
        self.data_path=self.PS.data_path

        self.flog = Experiment(self.data_path, prefix, liveplot_enabled=False)
        self.set_flog()

        print('_________________________')
        print('Monitoring Furnace at:')
        print('_________________________')
        print(self.flog.fname)
    

    def set_flog(self):
        with self.flog.datafile() as f:
            f.attrs.create('start_time', self.st, dtype='S20')
            v_string =special_dtype(vlen=bytes)
            f.attrs.create('Pressure_log', self.p_log, dtype=v_string)
            for keys in iter(self.furnace_config.keys()):
                f.attrs[keys] = self.furnace_config[keys]

    def update_log(self):
        while False:
            self.log_lock.acquire(blocking=False)
        with self.log_lock:
            self.fil_state=self.PS.read('fil_state')
            if self.fil_state==0:
                print('_________________________')
                print('Filament turned off, emergency shutdown')
                self.furnace_shut_down()
            else:
                pass
            if self.PS.server_state()==False:
                print('_________________________')
                print('Lost connection to pressure server')
                self.furnace_shut_down()
            with self.flog.datafile() as f:
                f.append('tpts', time.time())
                f.append('RGA_pres', self.PS.read('p_tot'))
                f.append('curr', self.xln.get_meas_current())
                f.append('volts', self.xln.get_meas_volt())
                for keys in self.pp_keys:
                    f.append_pt(keys, self.PS.read(keys))

    def _par_logger(self, interval=10):

        start_time=time.time()
        while not self.stop_logging:
            self.update_log()
            time.sleep(interval-(time.time()-start_time)%interval)


class Server():
    def __init__(self, config_path=None, launch_at_start=True):
        if config_path==None:
            local_path=os.getcwd()
            config_path=local_path+'\\furnace_config.yml'
        try:
            with open(config_path, 'r') as f:
                self.RGA_config=yaml.full_load(f)['RGA']
            self.data_path = self.RGA_config['data_path']
            if self.data_path==None:
                log_dir='\\Furnace Logs'
                self.data_path=os.path.dirname(config_path)+log_dir
                if not os.path.exists(self.data_path):
                    os.makedirs(self.data_path)
                self.data_path=self.data_path+'\\'
            
        except:
            print('Unable to find or open server, check config_path and config.')
        if launch_at_start:
            self.log, self.PID = self.pressure_server(self.data_path)
        else:
            pass

    def get_log(self):
        return self.log

    def get_PID(self):
        return self.PID

    def last_log_fname(self, path, prefix):
        dirlist = glob.glob(path + '*' + prefix + '*')
        dsort = dirlist.sort()
        i = 1
        max_iter = 100
        if dsort == None and len(dirlist) == 0:
            date = prefix.split('_')[0]
            while dsort == None and len(dirlist) == 0 and i < max_iter:
                prefix = (datetime.strptime(date, '%Y-%m-%d') - timedelta(days=i)).strftime('%Y-%m-%d') + '_RGA_log'
                dirlist = glob.glob(path + '*' + prefix + '*')
                dirlist.sort()
                i += 1
            if i == max_iter:
                return None
            else:
                f_name = os.path.split(dirlist[-1])[-1]
        else:
            f_name = os.path.split(dirlist[-1])[-1]
        return f_name

    def pressure_server(self, data_path):
        next_fname = self.next_log_fname(data_path)
        num = next_fname.split('_')[0]
        prefix = next_fname.split(num + '_')[1]
        last_fname = self.last_log_fname(data_path, prefix)
        if last_fname!=None:
            log_time = self.swmr_read(data_path + last_fname, 'tpts', -1)
            et = time.time() - log_time
            if et > 30:
                self.p_log, self.p_log_ID = self.start_pressure_server(data_path)
            else:
                try:
                    PID = self.swmr_attr(data_path + last_fname, 'LOG_PID')
                    p_name = self.process_name(PID)
                    if p_name == 'python.exe':
                        self.p_log = last_fname
                        self.p_log_ID = PID
                        print('Server Running:')
                        print('_________________')
                        print('Server Log: %s' % self.p_log)
                        print('Server Process ID %i' % self.p_log_ID)
                    else:
                        self.p_log, self.p_log_ID = self.start_pressure_server(data_path)
                except:
                    print('ERROR in determining PID from log')
        else:
            self.p_log, self.p_log_ID = self.start_pressure_server(data_path)
        return self.p_log, self.p_log_ID

    def server_state(self):
        p_name = self.process_name(self.PID)
        if p_name == 'python.exe':
            state = True
        else:
            state = False
        return state

    def read(self, key):
        data = self.swmr_read(self.data_path + self.log, key, -1)
        return data

    def attr_read(self, key):
        data=self.swmr_attr(self.data_path+self.log, key)
        return data

    def pp_keys(self):
        data_file = File(self.data_path + self.log, 'r', libver='latest', swmr=True)
        try:
            keys = [key for key in data_file.keys()]
            pp_key_vals = []
            for vals in keys:
                if vals != 'tpts' and vals != 'p_tot':
                    pp_key_vals.append(vals)
                else:
                    pass
            data_file.close()
            return pp_key_vals
        except:
            self.exit_handler()
            raise Exception('ERROR')

    def start_pressure_server(self, data_path, suffix='h5'):
        '''
        Starts an external pressure datalogger using shell command
        '''
        try:
            conf = subprocess.call('start python pressure_server.py', shell=True)
            if conf != 0:
                raise Exception('Error in shell script, check instrument')
            else:
                pass

            new_log_fname = self.next_log_fname(data_path)

            i = 0
            max_iter = 60
            print('Waiting for server to start...')

            while os.path.isfile(data_path + new_log_fname + '.' + suffix) == False and i < max_iter:
                time.sleep(1)
                i += 1
            if i >= max_iter:
                raise Exception('ERROR: Timed out waiting for log to be created')
            else:
                pass
            print('_________________')

            try:
                time.sleep(2)
                PID = self.swmr_attr(data_path + new_log_fname + '.' + suffix, 'LOG_PID')
                print('New log created: %s' % new_log_fname + '.' + suffix)
                print('Server Process ID: %i' % PID)
                return new_log_fname + '.' + suffix, PID
            except:
                raise Exception('ERROR: Server has no PID')

        except RuntimeError:
            print('Error occurred in server initialization')

    def exit_handler(self):
        '''
        Function that finds any instances of open H5 files and safely
        closes them in event of normal interpreter shutdown or error.
        '''
        for obj in gc.get_objects():  # Browse through ALL objects
            if isinstance(obj, File):  # Just HDF5 files
                try:
                    obj.close()
                except:
                    pass  # Was already closed

    def latest_file(self, filepath):
        list_files = glob.glob(filepath + '*')
        latest_file = max(list_files, key=os.path.getctime)
        data_file = latest_file.split('\\')[-1]
        return data_file

    def swmr_read(self, file_path, key, index):
        data_file = File(file_path, 'r', libver='latest', swmr=True)
        try:
            data = np.array(data_file[key])[index]
            data_file.close()
            return data
        except:
            self.exit_handler()

    def swmr_attr(self, file_path, key):
        data_file = File(file_path, 'r', libver='latest', swmr=True)
        try:
            attr_val = data_file.attrs[key]
            data_file.close()
            return attr_val
        except:
            self.exit_handler()
            print('ERROR')

    def kill(self):
        self.kill_server(self.PID)

    def process_name(self, PID):
        '''
        Finds process name for a given process ID using shell commands and parsing return.
        '''
        out = subprocess.check_output('tasklist /fi "pid eq {0}"'.format(PID), shell=True).decode("utf-8")
        if out.split(':')[0] == 'INFO':
            return 'no task'
        else:
            return out.split('\r\n')[3].split(' ')[0]

    def kill_server(self, PID):
        '''
        safely kills the data server via shell. Checks if process is a valid python executable.
        '''
        try:
            p_name = self.process_name(PID)
            if p_name == 'python.exe':
                # needs /t flag for safe process shutdown
                subprocess.Popen('taskkill /PID {0} /t'.format(PID), shell=True)
            elif p_name == 'no task':
                print('No process currently running with that ID')
            else:
                print('Not a valid python interpreter, cannot close process')
        except:
            print('ERROR Occurred in killing process')

    def _get_next_filename(self, datapath,prefix,suffix=''):
        ii = self._next_file_index(datapath, prefix)
        return "%05d_" % (ii) + prefix +suffix

    def _next_file_index(self, datapath,prefix=''):
        """Searches directories for files of the form *_prefix* and returns next number
            in the series"""

        dirlist=glob.glob(os.path.join(datapath,'*_'+prefix+'*'))
        dirlist.sort()
        try:
            ii=int(os.path.split(dirlist[-1])[-1].split('_')[0])+1
        except:
            ii=0
        return ii

    def next_log_fname(self, path):
        local_time = time.time()
        date = datetime.fromtimestamp(local_time).strftime('%Y-%m-%d %H:%M:%S').split(' ')[0]
        prefix = date.split(' ')[0] + '_RGA_log'
        next_file = self._get_next_filename(path, prefix)
        return next_file

class FurnaceAutomation(FurnaceSetup):
        def __init__(self, config_path=None, system_config_path=None, kill_server=False, p_server=False):
            super(FurnaceAutomation, self).__init__(config_path, system_config_path, kill_server, p_server)
            self.ramp_interval=2
        
        def start_log(self, parallel=False):
            self.stop_logging=False
            if [names for names in self._get_job_names() if 'Logger' in names]==[]:
                if parallel==True:
                    self._thread_check()
                    thread=self._set_thread('Logger', self._par_logger, self.log_interval)
                    thread.start()
                    if self.jobs[-1].is_alive():
                        print('_________________________')
                        print('Logger running')
                    else:
                        raise Exception('Unable to start the nitrogen thread')
                    return thread
                else:
                    self._par_logger(self.log_interval)
            return

        def start_ramping(self):
            if not self.xln.get_state() or self.xln.get_meas_volt()<self.min_volts:
                self.start_supply()

            state_dict=[{'hold':(self.PH_time,), 'name':'pre-heat'},
                        {'ramp':(self.min_volts, self.Hold_v, self.ramp_up_rate, self.ramp_interval), 'name':'ramp-up'},
                        {'hold':(self.Hold_time,), 'name':'soak'},
                        {'ramp':(self.Hold_v, self.min_volts, self.ramp_down_rate, self.ramp_interval), 'name':'ramp-down'}
                        ]
            
            self.xln.ramp_profile(state_dict=state_dict, parallel=True)
            self.jobs.append(self.xln.jobs[-1])

            if self.jobs[-1].is_alive():
                print('_________________________')
                print('Ramping started, in state: %s'%self.xln.ramp_state)
                self.automation_state=self.xln.ramp_state
            else:
                raise Exception('Unable to start ramping, check instruments...')

        def nitrogen_backfill(self, parallel=False, delay=False):
            self.stop_nitrogen=False
            if delay==False:
                delay=0
            else:
                delay=self.nitrogen_start_time
            
            if [names for names in self._get_job_names() if 'NitrogenFill' in names]==[]:
                if parallel==True:
                    self._thread_check()
                    thread=self._set_thread('NitrogenFill', self._par_delayed_nitrogen_backfill, delay, self.nitrogen_hold_time, 'soak')
                    self.VAC_jobs.append(thread)
                    thread.start()
                    if self.jobs[-1].is_alive():
                        print('_________________________')
                        print('Nitrogen armed and holding')
                    else:
                        raise Exception('Unable to start the nitrogen thread')

                    return thread
                else:
                    self._par_delayed_nitrogen_backfill(delay, self.nitrogen_hold_time, 'soak')

            
        def _par_delayed_nitrogen_backfill(self, delay, fill_time, start_cond='soak',):
            if [names for names in self._get_job_names() if 'VRamp' in names]==[]:
                return 
            
            while self.xln.ramp_state!=start_cond:
                pass
            
            et=0
            start_time=time.time()
            while et<=delay and not self.stop_nitrogen:
                et=time.time()-start_time
                pass

            self._par_nitrogen_backfill(fill_time)

        def _par_backfill_test(self, delay, fill_time, start_cond):
            if [names for names in self._get_job_names() if 'VRamp' in names]==[]:
                return 

            while self.xln.ramp_state!=start_cond:
                pass
            
            et=0
            start_time=time.time()
            while et<=delay and not self.stop_nitrogen:
                et=time.time()-start_time
                pass

            self.stop_nitrogen=True



        def start_supply(self):
            try:
                self.xln.get_id()
            except:
                raise Exception('ERROR in communicating with power supply')

            self.xln.set_state(False)
            self.xln.set_voltage_limit(self.max_volts)
            self.xln.set_volt(self.min_volts)
            self.xln.set_current(self.max_current)
            self.xln.set_state(True)
            i=0
            max_iter=10
            while self.xln.get_meas_volt()<11.9 and i<=max_iter:
                self.xln.set_state(False)
                time.sleep(1)
                self.xln.set_state(True)
                i+=1
            if i>=max_iter:
                self.xln.set_state(False)
                print('_________________________')
                print('Power supply unable to reach starting voltage, exiting automation')
                exit()
            else:
                pass
            print('_________________________')
            print('Power supply on, furnace started')




In [5]:
furnace=FurnaceAutomation()

_________________________
Ramp up rate: 0.01
Ramp down rate: 0.01
Pre heat time: 30
P hold: 3e-05
P start: 6e-06
Hold v: 2.0
Hold time: 300
logging interval: 5
_________________________


In [6]:
furnace._par_nitrogen_backfill(75)

1 [0, 1, 0, 0, 0, 0, 0, 0]
2 [0, 1, 0, 0, 0, 0, 0, 0]


KeyboardInterrupt: 

In [10]:
furnace.get_valves()

[0, 1, 0, 0, 0, 0, 0, 0]

In [17]:
# furnace.start_ramping()
# furnace.nitrogen_backfill(parallel=True, delay=True)

# t_int=5
# start_time=time.time()
# while furnace.jobs[0].is_alive():
#     print(furnace.xln.ramp_state, furnace.stop_nitrogen, time.time()-start_time)
#     time.sleep(t_int-(time.time()-start_time)%t_int)
clear

NameError: name 'clear' is not defined

In [32]:
furnace.set_open('RGA')
furnace.set_close('ROUGHING')
furnace.set_open('BACKING')
furnace.set_valves()
furnace.set_open('GATE')
furnace.set_valves()

In [48]:
furnace.get_valves()

[1, 0, 1, 0, 1, 0, 0, 0]

In [46]:
_par_nitrogen_backfill(furnace, 75)

3 [1, 0, 1, 0, 1, 0, 0, 0]


In [45]:
def _par_nitrogen_backfill(self, f_time):
        f_time=int((f_time-(f_time-60)%60)/60)
        if self._check_for_alive(jobs=self.VAC_jobs):
            return 
        self.set_close('RGA')
        self.set_valves()
        self.set_close('GATE')
        self.set_open('ROUGHING')
        self.set_valves()
        if self.PLC.get_state(5)[1]==False:
            self.PLC.set_pulse_time(f_time)
            self.PLC.pulse(self.valves['PULSE']['CHANNEL'])
            et=0
            start_time=time.time()
            while et<=f_time*60 or not self.P_monitor.set_point_status()[5]:
                et=time.time()-start_time
                pass
            self.set_open('GATE')
            self.set_close('ROUGHING')
            self.set_valves()
            while not self.P_monitor.set_point_status()[0]:
                pass
            time.sleep(.5)
            self.set_open('RGA')
            self.set_valves()

In [29]:
furnace.PLC.v_query('RESET:')

'RESET:SUCCESS'

In [5]:
state_dict=[{'hold':(30,)}, {'ramp':(.2, 2, .01, 2)}, {'hold':(200,)}, {'ramp':(2, .1, .01, 2)}]

furnace.start_log(parallel=True)
furnace.xln.set_volt(.2)
furnace.xln.ramp_profile(state_dict=state_dict, parallel=True)
furnace.jobs.append(furnace.xln.jobs[-1])

print(furnace.jobs)
# print_int=5
# i=1
# start_time=time.time()
# while furnace.jobs[-1].is_alive():
#     print(furnace.xln.ramp_state, furnace.jobs[0].is_alive())
#     if i==20:
#         furnace.xln.hold=True
#     elif i==30:
#         furnace.xln.hold=False
#     time.sleep(print_int-(time.time()-start_time)%print_int)
#     i+=1




[<Thread(Logger, started 1416)>, <Thread(VRamp, started 16284)>]


In [6]:
# print_int=5
# i=1
# start_time=time.time()
# while furnace.jobs[-1].is_alive():
#     print(furnace.xln.ramp_state, furnace.jobs[0].is_alive())
#     if i==20:
#         furnace.xln.hold=True
#     elif i==30:
#         furnace.xln.hold=False
#     time.sleep(print_int-(time.time()-start_time)%print_int)
#     i+=1
1+1

2

In [7]:
furnace.jobs

[]

In [7]:
furnace.stop_logging=True
furnace.xln.stop_ramp=True
time.sleep(5)
furnace.jobs

[<Thread(Logger, stopped 1416)>, <Thread(VRamp, stopped 16284)>]

In [11]:
furnace.jobs

[<Thread(Logger, stopped 972)>, <Thread(VRamp, stopped 7640)>]

In [56]:
class primary():
    def __init__(self, a):
        self.a=a

    def adder(self):
        self.a+=1
        self.test_trial()
        return self.a

class secondary(primary):
    def __init__(self, a):
        primary.__init__(self, a)

    def test_trial(self):
        if self.a>3:
            print('hey')

In [57]:
sec=secondary(1)

In [70]:
threading.enumerate()

[<_MainThread(MainThread, started 19336)>,
 <Thread(Thread-2, started daemon 8696)>,
 <Heartbeat(Thread-3, started daemon 12428)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 19684)>,
 <ParentPollerWindows(Thread-1, started daemon 17416)>]

In [21]:

int((t-(t-60)%60)/60)

10